In [3]:
import datamol as dm
import polaris as po

from time import time

# SMILES to 3D representations of molecules

Let's find out how to go from SMILES to 3D, using `datamol`package. Then run some timing tests

In [4]:
# Get the data
competition = po.load_competition(f"asap-discovery/antiviral-potency-2025")
competition.cache()
train, test = competition.get_train_test_split()

[2025-02-25 16:12:43] INFO     The version of Polaris that was used to create the artifact (0.11.0) ]8;id=625181;file:///home/stas/.mamba/envs/polaris-env/lib/python3.12/site-packages/polaris/_artifact.py\_artifact.py]8;;\:]8;id=630741;file:///home/stas/.mamba/envs/polaris-env/lib/python3.12/site-packages/polaris/_artifact.py#92\92]8;;\
                               is different from the currently installed version of Polaris                        
                               (0.11.8).                                                                           

                      WARNING  You're loading data from a remote location. If the dataset is small     ]8;id=859966;file:///home/stas/.mamba/envs/polaris-env/lib/python3.12/site-packages/polaris/dataset/_base.py\_base.py]8;;\:]8;id=85873;file:///home/stas/.mamba/envs/polaris-env/lib/python3.12/site-packages/polaris/dataset/_base.py#181\181]8;;\
                               enough, consider caching the dataset first using                                    
                               CompetitionSpecification.cache() for more performant data access.                   

Output()

[16:12:44] Copying to destination                                                                      ]8;id=974767;file:///home/stas/.mamba/envs/polaris-env/lib/python3.12/site-packages/polaris/dataset/_base.py\_base.py]8;;\:]8;id=443910;file:///home/stas/.mamba/envs/polaris-env/lib/python3.12/site-packages/polaris/dataset/_base.py#380\380]8;;\
           /home/stas/.cache/polaris/datasets/7e25635c-61bd-4097-9006-63f390a78be5/data.zarr                       

           For large Zarr archives, this may take a while.                                             ]8;id=775042;file:///home/stas/.mamba/envs/polaris-env/lib/python3.12/site-packages/polaris/dataset/_base.py\_base.py]8;;\:]8;id=131618;file:///home/stas/.mamba/envs/polaris-env/lib/python3.12/site-packages/polaris/dataset/_base.py#381\381]8;;\

[16:12:45]  Success: Copying Zarr keys                                                                ]8;id=332074;file:///home/stas/.mamba/envs/polaris-env/lib/python3.12/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=189933;file:///home/stas/.mamba/envs/polaris-env/lib/python3.12/site-packages/polaris/utils/context.py#53\53]8;;\

            Success: Copying Zarr archive                                                             ]8;id=335291;file:///home/stas/.mamba/envs/polaris-env/lib/python3.12/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=110796;file:///home/stas/.mamba/envs/polaris-env/lib/python3.12/site-packages/polaris/utils/context.py#53\53]8;;\

In [ ]:

# TODO: think of a realistic rms cutoff (ask Slav) and energy iterations
def generate_conformers(
        smiles: str, 
        num_confs: int = 5, 
        minimize_energy: bool = False, 
        rms_cutoff: int = None,
        energy_iterations: int = 1000
    ) -> dm.Mol:
    """
    Generate 3D conformers using datamol.
    """
    mol = dm.to_mol(smiles, add_hs=True, sanitize=True)
    mol_confs = dm.conformers.generate(
        mol, 
        n_confs=num_confs,
        num_threads=8,
        minimize_energy=minimize_energy, 
        rms_cutoff=rms_cutoff,
        energy_iterations=energy_iterations
    )
    return mol_confs

In [9]:
def run_conformer_test(n_examples, minimize_energy=True, energy_iterations=100):
    start = time()
    for num, (x, _) in enumerate(train):
        if num < n_examples:
            mol_confs = generate_conformers(
                x, num_confs=10, 
                minimize_energy=minimize_energy,
                energy_iterations=energy_iterations
        )
        else:
            break

    if minimize_energy:
        energy_minimization_str = 'with energy minimization for ' + str(energy_iterations) + ' steps:\n' 
    else:
        energy_minimization_str = 'without energy minimization:\n'

    message = (
        f"Generating 10 conformers for {n_examples} examples "
        f"{energy_minimization_str}"
        f"time spent: {(time() - start):.4} seconds\n"
    )
    return message

In [15]:
# Run some tests on 10, 50 and 100 data points and time it.
# WARNING: this will run for about 3 mins
for n in [10, 50, 100]:
    print(run_conformer_test(n, False))
    print(run_conformer_test(n, True, 100))
    print(run_conformer_test(n, True, 1000))
    print(80 * "*")

Generating 10 conformers for 10 examples without energy minimization:
time spent: 2.518 seconds

Generating 10 conformers for 10 examples with energy minimization for 100 steps:
time spent: 3.052 seconds

Generating 10 conformers for 10 examples with energy minimization for 1000 steps:
time spent: 3.897 seconds

********************************************************************************
Generating 10 conformers for 50 examples without energy minimization:
time spent: 12.57 seconds

Generating 10 conformers for 50 examples with energy minimization for 100 steps:
time spent: 16.67 seconds

Generating 10 conformers for 50 examples with energy minimization for 1000 steps:
time spent: 23.71 seconds

********************************************************************************
Generating 10 conformers for 100 examples without energy minimization:
time spent: 27.87 seconds

Generating 10 conformers for 100 examples with energy minimization for 100 steps:
time spent: 36.42 seconds

Gen

## Conclusions

- Time scales roughly linearly as expected
- It takes almost twice as long to generate conformers with 1000 minimization steps in comparison to no energy 
minimization approach.
- It's around 40% slower to generate conformers with 1000 steps of energy minimization VS 100 steps.
- **We can easily either preprocess all molecules and store their various conformers or we can cache the call to the 
generation function directly in the data pipeline to avoid further recomputation** I would go for the 1st option, since
it'll be faster to iterate over training attempts, however, we would need to include that preprocessing step into the 
test data pipeline. (Let's discuss it)

## Check how a 3D conformation looks like

In [19]:
# Print 3D coords of the 1st conformer
mol_confs = generate_conformers(next(train)[0])
conf_coords = mol_confs.GetConformer(0).GetPositions()

print(f"Shape: {conf_coords.shape}\n")
print(f"Coords: \n {conf_coords}")

Shape: (39, 3)

Coords: 
 [[-3.96347098 -2.55323728  4.68226565]
 [-3.75070309 -2.99703939  3.31937879]
 [-2.55738507 -2.67011056  2.61592369]
 [-1.706712   -1.98581986  3.22614405]
 [-2.42670803 -3.16453177  1.23251934]
 [-1.17116901 -2.78020459  0.58939718]
 [-1.13109317 -1.57789599 -0.1536081 ]
 [ 0.23291616 -1.06035595 -0.49371585]
 [ 0.24953444 -0.47632158 -1.86391284]
 [-0.19000778 -1.10716496 -2.87080904]
 [ 0.80509016  0.81858624 -1.84288783]
 [ 0.91438288  1.64311    -2.99648605]
 [ 2.06042671  1.55290008 -3.76242247]
 [ 2.26032263  2.2919627  -4.87474   ]
 [ 1.31160754  3.18151778 -5.29761092]
 [ 0.15781061  3.29536168 -4.5557934 ]
 [-0.81229288  4.18673132 -4.96401498]
 [-1.98430021  4.33285963 -4.25020975]
 [-2.15629249  3.55294494 -3.10617703]
 [-1.20008763  2.6553742  -2.67925591]
 [-0.04908513  2.52989773 -3.40354378]
 [ 1.2118446   1.09744302 -0.49336224]
 [ 0.48654999  0.13870023  0.35408722]
 [ 1.19165993 -0.19286239  1.64960681]
 [ 1.3800087   1.06398235  2.405178  ]